In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from shapely import wkt

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  set_use_pygeos()
/tmp/ipykernel_76541/72832894.py:5: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


2024-01-25 16:34:24 Configured OSMnx 1.1.2
2024-01-25 16:34:24 HTTP response caching is on


In [2]:
# Query municipalities
mun_schema = 'marco'
mun_table = 'mpos_2020'
query = f"SELECT * FROM {mun_schema}.{mun_table} "
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Create Mexico shape
mun_gdf = mun_gdf.to_crs("EPSG:6372")
mun_buffer = mun_gdf.buffer(1)
mun_gdf_buffer = gpd.GeoDataFrame(mun_buffer)
mun_gdf_buffer.rename(columns={0:'geometry'},inplace=True)
mexico = mun_gdf_buffer.dissolve()
mexico = mexico.to_crs("EPSG:4326")

# Show
mexico.plot()

OperationalError: (psycopg2.OperationalError) could not translate host name "None" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [1]:
cell = '85d'
#next_cells = []
#done_cells = ['80d','813','86d','815','83f','841','85b','86f','865','86b','8f5'(2522s),
# '867','869','85f','859','843']
#failed_cells = []

chunksize = 50000000

directory = f"../../../data/external/building_footprints/{cell}_buildings.csv"
aup.log(f'Started reading file for cell {cell}')

cont = 1

for buildings_tmp in pd.read_csv(directory, chunksize=chunksize):
    # process each chunk here
    aup.log(f'Finished reading file for iteration {cont}')

    buildings_tmp['geometry'] = buildings_tmp['geometry'].apply(wkt.loads)
    buildings_gdf = gpd.GeoDataFrame(buildings_tmp, crs='epsg:4326')
    del buildings_tmp
    buildings_gdf['cell'] = cell

    aup.log('Finished assigning geometry')

    limit_len = 5000000
    if len(buildings_gdf)>limit_len:
        c_upload = len(buildings_gdf)/limit_len
        for k in range(int(c_upload)+1):
            aup.log(f"Starting range k = {k} of {int(c_upload)}")
            gdf_inter_upload = buildings_gdf.iloc[int(limit_len*k):int(limit_len*(1+k))].copy()
            aup.gdf_to_db_slow(gdf_inter_upload,'buildings_google_v3','google_buildings',if_exists="append")
    else:
        aup.log('Starting upload of all data')
        aup.gdf_to_db_slow(buildings_gdf,'buildings_google_v3','google_buildings',if_exists="append")

    cont += 1

    del buildings_gdf
    del gdf_inter_upload

NameError: name 'aup' is not defined

In [ ]:
mexico_buildings = gpd.sjoin(buildings_gdf,mexico,how='inner')

# Show
print(mexico_buildings.shape)
mexico_buildings.head(2)